In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2

This library is intended to be used as an alternative to `pd.Series.rolling` and `pd.Series.expanding` to gain a speedup by using numba optimized functions operating on numpy arrays. There are also online classes for more efficient updates of window statistics.

## Install

`pip install window-ops`

## How to use

### Transformations

For a transformations `n_samples` -> `n_samples` you can use `{[seasonal_](rolling|expanding)}_{(mean|max|min|std)}` on an array.

#### Benchmarks

In [ ]:
#| include: false
import random
import time
from functools import partial

import numpy as np
import pandas as pd
from IPython.display import Markdown

import window_ops
from window_ops.ewm import *
from window_ops.expanding import *
from window_ops.online import *
from window_ops.rolling import *

In [ ]:
pd.__version__

'1.2.3'

In [ ]:
n_samples = 1_000  # array size
window_size = 8  # for rolling operations
season_length = 7  # for seasonal operations
execute_times = 1_000 # number of times each function will be executed

In [ ]:
#| include: false
np.random.seed(0)
random.seed(0)
y = np.random.rand(n_samples)
ys = pd.Series(y)
groups = np.arange(n_samples) % season_length
grouped_y = ys.groupby(groups)

In [ ]:
#| include: false
times = {'window_ops': {}, 'pandas': {}}
ops = ('mean', 'max', 'min', 'std')
for kind in ('rolling', 'expanding', 'seasonal_rolling', 'seasonal_expanding'):
    for op in ops:
        functions = {}
        if kind == 'rolling':
            functions['window_ops'] = partial(getattr(window_ops.rolling, f'{kind}_{op}'), window_size=window_size)
            functions['pandas'] = lambda y: y.rolling(window_size, min_periods=window_size).agg(op)
        elif kind == 'expanding':
            functions['window_ops'] = getattr(window_ops.expanding, f'{kind}_{op}')
            functions['pandas'] = lambda y: y.expanding().agg(op)
        elif kind == 'seasonal_rolling':
            functions['window_ops'] = partial(getattr(window_ops.rolling, f'{kind}_{op}'), 
                                              season_length=season_length, window_size=window_size)
            functions['pandas'] = lambda grouped_y: grouped_y.transform(lambda x: x.rolling(window_size).agg(op))
        elif kind == 'seasonal_expanding':
            functions['window_ops'] = partial(getattr(window_ops.expanding, f'{kind}_{op}'), season_length=season_length)
            functions['pandas'] = lambda grouped_y: grouped_y.transform(lambda x: x.expanding().agg(op))
        else:
            raise ValueError(kind)
            
        results = {}            
        for impl in ('window_ops', 'pandas'):
            if impl == 'window_ops':
                x = y
            else:
                x = grouped_y if kind.startswith('seasonal') else ys
            func = functions[impl]
            results[impl] = func(x)
            start = time.perf_counter()
            for _ in range(execute_times):
                func(x)
            times[impl][f'{kind}_{op}'] = time.perf_counter() - start
        assert np.allclose(results['window_ops'], results['pandas'], equal_nan=True)
times = pd.DataFrame(times) * 1_000 / execute_times

In [ ]:
#| include: false
def display_dataframe(df, fmt):
    return Markdown(df.applymap(fmt.format).to_markdown())

Average times in milliseconds.

In [ ]:
display_dataframe(times, fmt='{:.2f}')

|                         |   window_ops |   pandas |
|:------------------------|-------------:|---------:|
| rolling_mean            |         0    |     0.17 |
| rolling_max             |         0.01 |     0.19 |
| rolling_min             |         0.01 |     0.23 |
| rolling_std             |         0.01 |     0.22 |
| expanding_mean          |         0    |     0.13 |
| expanding_max           |         0    |     0.13 |
| expanding_min           |         0    |     0.13 |
| expanding_std           |         0.01 |     0.14 |
| seasonal_rolling_mean   |         0    |     2.62 |
| seasonal_rolling_max    |         0.02 |     3.04 |
| seasonal_rolling_min    |         0.02 |     2.85 |
| seasonal_rolling_std    |         0.01 |     2.37 |
| seasonal_expanding_mean |         0    |     1.9  |
| seasonal_expanding_max  |         0.01 |     1.79 |
| seasonal_expanding_min  |         0.01 |     1.81 |
| seasonal_expanding_std  |         0.01 |     2.45 |

In [ ]:
#| include: false
speedups = times['pandas'] / times['window_ops']
speedups = speedups.to_frame('times faster')

In [ ]:
display_dataframe(speedups, fmt='{:.0f}')

|                         |   times faster |
|:------------------------|---------------:|
| rolling_mean            |             76 |
| rolling_max             |             14 |
| rolling_min             |             21 |
| rolling_std             |             33 |
| expanding_mean          |             44 |
| expanding_max           |             32 |
| expanding_min           |             32 |
| expanding_std           |             19 |
| seasonal_rolling_mean   |            632 |
| seasonal_rolling_max    |            201 |
| seasonal_rolling_min    |            173 |
| seasonal_rolling_std    |            322 |
| seasonal_expanding_mean |            494 |
| seasonal_expanding_max  |            353 |
| seasonal_expanding_min  |            339 |
| seasonal_expanding_std  |            238 |

### Online

If you have an array for which you want to compute a window statistic and then keep updating it as more samples come in you can use the classes in the `window_ops.online` module. They all have a `fit_transform` method which take the array and return the transformations defined above but also have an `update` method that take a single value and return the new statistic.

In [ ]:
#| include: false
times = {}
ops = ('mean', 'max', 'min', 'std')
online_class = None
for kind in ('Rolling', 'Expanding', 'SeasonalRolling', 'SeasonalExpanding'):
    for op in ops:
        if kind == 'Rolling':
            online_class = getattr(window_ops.online, f'{kind}{op.title()}')(window_size=window_size)
        elif kind == 'Expanding':
            online_class = getattr(window_ops.online, f'{kind}{op.title()}')()
        elif kind == 'SeasonalRolling':
            online_class = getattr(window_ops.online, f'{kind}{op.title()}')(season_length=season_length, window_size=window_size)
        elif kind == 'SeasonalExpanding':
            online_class = getattr(window_ops.online, f'{kind}{op.title()}')(season_length=season_length)
        else:
            raise ValueError(kind)
        
        # compile
        online_class.fit_transform(y)
        
        start = time.perf_counter()
        for _ in range(execute_times):
            online_class.fit_transform(y)
            for i in range(100):
                online_class.update(i)
        times[f'{kind}{op.title()}'] = time.perf_counter() - start
times = pd.Series(times, name='average time (ms)') * 1_000 / execute_times

#### Benchmarks

Average time in milliseconds it takes to transform the array and perform 100 updates.

In [ ]:
display_dataframe(times.to_frame(), '{:.2f}')

|                       |   average time (ms) |
|:----------------------|--------------------:|
| RollingMean           |                0.07 |
| RollingMax            |                0.09 |
| RollingMin            |                0.09 |
| RollingStd            |                0.24 |
| ExpandingMean         |                0.08 |
| ExpandingMax          |                0.03 |
| ExpandingMin          |                0.02 |
| ExpandingStd          |                0.08 |
| SeasonalRollingMean   |                0.18 |
| SeasonalRollingMax    |                0.14 |
| SeasonalRollingMin    |                0.19 |
| SeasonalRollingStd    |                0.25 |
| SeasonalExpandingMean |                0.09 |
| SeasonalExpandingMax  |                0.06 |
| SeasonalExpandingMin  |                0.06 |
| SeasonalExpandingStd  |                0.09 |